In [17]:
import pandas as pd
import re
import os

def extract_sensor_data(file_path):
    if not os.path.exists(file_path):
        return "Error: File path not found."

    with open(file_path, 'r') as f:
        content = f.read()

    # Regex to capture the 8 data columns [cite: 2, 3, 4]
    pattern = re.compile(
        r"(\d+)\s*\|\s*(\d+)\s*\|\s*([-.\d]+)\s*\|\s*([\d.]+)\s*\|\|\s*"
        r"([\d-]+|---)\s*\|\s*([\d-]+|---)\s*\|\s*([-.\d]+|---)\s*\|\s*([\d.]+|---)"
    )

    matches = pattern.findall(content)
    
    # Process rows and handle the '---' null values [cite: 89, 90, 91]
    rows = []
    for m in matches:
        row = [val if val != "---" else None for val in m]
        rows.append(row)

    # Define temporary columns
    columns = ["Pk_Idx", "Pk_Loc", "Pk_Sens", "Pk_R2", "Vl_Idx", "Vl_Loc", "Vl_Sens", "Vl_R2"]
    full_df = pd.DataFrame(rows, columns=columns)
    
    # Convert all columns to numeric, forcing errors to NaN [cite: 5, 7, 55]
    full_df = full_df.apply(pd.to_numeric, errors='coerce')

    # Split into two separate DataFrames
    # DF 1: Peaks (Columns 1-4)
    df_peaks = full_df[["Pk_Idx", "Pk_Loc", "Pk_Sens", "Pk_R2"]].copy()
    
    # DF 2: Valleys (Columns 5-8)
    # We drop rows where Vl_Idx is NaN to keep the valley table clean [cite: 89, 92]
    df_valleys = full_df[["Vl_Idx", "Vl_Loc", "Vl_Sens", "Vl_R2"]].dropna(subset=["Vl_Idx"]).copy()

    return df_peaks, df_valleys




In [18]:
# Target File Path
path = r"C:\Users\User\Documents\Lyubov\MODIFICATION of probe signature\chan_4_sens_LV54_Filter_Buttherworth_filter_1000Hz_log.txt"

# Generate the two DataFrames
df_peaks, df_valleys = extract_sensor_data(path)

# Individual Arrays can still be accessed easily:
pk_sensitivity_array = df_peaks["Pk_Sens"].values
vl_sensitivity_array = df_valleys["Vl_Sens"].values

print("--- Peaks DataFrame ---")
print(df_peaks.head())

print("\n--- Valleys DataFrame ---")
print(df_valleys.head())


--- Peaks DataFrame ---
   Pk_Idx  Pk_Loc  Pk_Sens   Pk_R2
0       3     230 -32.5428  0.9611
1       6    1349 -41.3949  0.9854
2       7    1785 -30.6216  0.9817
3       8    2289 -26.9314  0.9750
4       9    2600 -34.3131  0.9634

--- Valleys DataFrame ---
   Vl_Idx  Vl_Loc   Vl_Sens   Vl_R2
0     5.0   887.0  -79.4568  0.9607
1     6.0  1205.0 -116.8115  0.9715
2     7.0  1565.0 -106.1490  0.9285
3     8.0  2126.0  -89.4367  0.9353
4     9.0  2467.0 -130.6223  0.9917
